In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
browser = webdriver.Chrome(executable_path='./chromedriver.exe')
#browser.maximize_window()

**Hàm kiểm tra "text" có phải là số không**

In [3]:
def is_number(text):
    for i in text:
        if i.isalpha():
            return False
    return True

**Hàm kiểm tra element có tồn tại không**

In [4]:
def check_element_exist(class_name):
    try:
        #aqi = driver.find_element_by_class_name('aqi-value__value')
        aqi = browser.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

In [5]:
city_names = []
status_text = []
PM25 = []
PM10 = []
O3 = []
NO2 = []
SO2 = []
CO = []

# Lấy data mục "Air quality and pollution city ranking"

In [6]:
url = browser.get('https://www.iqair.com/world-air-quality-ranking')
html_text = browser.page_source

m = browser.find_elements_by_class_name('over-x-auto')

m2 = browser.find_elements(By.XPATH, '//button')
for i in m2:
    if i.text == 'More':
        i.click()
        break


In [13]:
html_text = browser.page_source
tree = BeautifulSoup(html_text, 'html.parser')

links = tree.find_all('td',{'class' : 'text-left'})

urls = []
for link in links:
    urls.append('https://www.iqair.com'+link.find('a',{'class' : 'link-primary'})['href'])
#urls #list url 96 thành phố

In [14]:
for url in urls:
    browser.get(url)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    title = tree.find('h1',{'class' : 'pagetitle__title ng-star-inserted'}).text
    city = title[15:len(title)]
    print("City: ", city)
    city_names.append(city)
    
    index = tree.find_all('div', {'class': 'pollutant-history-type-buttons ng-star-inserted'})
    
    for j in index:
        
        x = browser.find_elements_by_class_name('mat-button-wrapper')
        for X in x:
            if 'DAILY' in X.text:
                X.click()
                break
    #lay data
        if not check_element_exist('aqi-status__text'):
            status_text.append('NULL')
            PM25.append('NULL')
            PM10.append('NULL')
            O3.append('NULL')
            NO2.append('NULL')
            SO2.append('NULL')
            CO.append('NULL')
        else:
            status = browser.find_element_by_class_name('aqi-status__text')
            status_text.append(status.text)
            print('Level: ', status.text)
            #lay ra danh sach cac button chi so
            button_board = browser.find_element_by_class_name('pollutant-history-type-buttons')
            buttons = button_board.find_elements_by_tag_name('button')
            #bo button AQI
            buttons = buttons[1:]
            for button in buttons:
                print('Button name: ', button.text)
                #click vao tung button
                button.click()
                #lay chi so cuoi cung cua bieu do trong button
                chart = browser.find_element_by_class_name('highcharts-series-group')
                cols = chart.find_elements_by_tag_name('rect')
                element_to_hover = cols[-1]
                ActionChains(browser).move_to_element(element_to_hover).perform()
                values = browser.find_elements_by_tag_name('tspan')
                if len(values) == 1:
                    index_value = 0
                else:
                    index_value = values[-2].text
                    if not is_number(index_value):
                        index_value = 0
                print('Value: ', index_value)
                
                if button.text == 'PM2.5':
                    PM25.append(index_value)
                elif button.text =='PM10':
                    PM10.append(index_value)
                elif button.text == 'O3':
                    O3.append(index_value)
                    print('append')
                elif button.text == 'NO2':
                    NO2.append(index_value)
                elif button.text == 'SO2':
                    SO2.append(index_value)
                elif button.text == 'CO':
                    CO.append(index_value)
                
            len_status_text = len(status_text)
            if len(PM25) < len_status_text:
                PM25.append('NULL')
            if len(PM10) < len_status_text:
                PM10.append('NULL')
            if len(O3) < len_status_text:
                O3.append('NULL')
            if len(NO2) < len_status_text:
                NO2.append('NULL')
            if len(SO2) < len_status_text:
                SO2.append('NULL')
            if len(CO) < len_status_text:
                CO.append('NULL')
    print("\n")
print('Finish')

City:  Delhi
Level:  Hazardous
Button name:  PM2.5
Value:  337.4
Button name:  PM10
Value:  483.6
Button name:  O3
Value:  31.8
append
Button name:  NO2
Value:  67.4
Button name:  SO2
Value:  22.2
Button name:  CO
Value:  2880.9


City:  Bishkek
Level:  Hazardous
Button name:  PM2.5
Value:  286.5
Button name:  PM10
Value:  2.2


City:  Lahore
Level:  Hazardous
Button name:  PM2.5
Value:  227.4


City:  Dhaka
Level:  Very Unhealthy
Button name:  PM2.5
Value:  148.2


City:  Guangzhou
Level:  Very Unhealthy
Button name:  PM2.5
Value:  118.1
Button name:  PM10
Value:  201.7
Button name:  O3
Value:  56.9
append
Button name:  NO2
Value:  172
Button name:  SO2
Value:  13.8
Button name:  CO
Value:  1487.5


City:  Ulaanbaatar
Level:  Very Unhealthy
Button name:  PM2.5
Value:  100.7
Button name:  PM10
Value:  105.5
Button name:  O3
Value:  2.3
append
Button name:  NO2
Value:  28.7
Button name:  SO2
Value:  89.5
Button name:  CO
Value:  24.5


City:  Hangzhou
Level:  Very Unhealthy
Button name:

Value:  13.4
Button name:  PM10
Value:  16.7
Button name:  O3
Value:  14.7
append
Button name:  NO2
Value:  34
Button name:  SO2
Value:  2.7
Button name:  CO
Value:  900


City:  Wroclaw
Level:  Moderate
Button name:  PM2.5
Value:  31.8
Button name:  PM10
Value:  3.8
Button name:  O3
Value:  25.1
append
Button name:  NO2
Value:  28.2
Button name:  SO2
Value:  4.7
Button name:  CO
Value:  523.3


City:  Oslo
Level:  Moderate
Button name:  PM2.5
Value:  15.3
Button name:  PM10
Value:  16.9
Button name:  NO2
Value:  41.1
Button name:  CO
Value:  0


City:  Taipei
Level:  Moderate
Button name:  PM2.5
Value:  23.3
Button name:  PM10
Value:  37.1
Button name:  O3
Value:  52.9
append
Button name:  NO2
Value:  70.9
Button name:  SO2
Value:  6.6
Button name:  CO
Value:  964.7


City:  Kyiv
Level:  Moderate
Button name:  PM2.5
Value:  20.1


City:  Paris
Level:  Moderate
Button name:  PM2.5
Value:  16.4
Button name:  PM10
Value:  21.7
Button name:  O3
Value:  28.8
append
Button name:  NO2
Value:

Value:  34.8
append
Button name:  NO2
Value:  15.5
Button name:  SO2
Value:  1.8


City:  Portland
Level:  Good
Button name:  PM2.5
Value:  6.2
Button name:  O3
Value:  2.4
append
Button name:  NO2
Value:  2.6
Button name:  SO2
Value:  0.7
Button name:  CO
Value:  17.1


City:  Vancouver BC
Level:  Good
Button name:  PM2.5
Value:  3.2
Button name:  PM10
Value:  4.2
Button name:  O3
Value:  30
append
Button name:  NO2
Value:  24.7
Button name:  SO2
Value:  1
Button name:  CO
Value:  251.2


City:  Salt Lake City
Level:  Good
Button name:  PM2.5
Value:  1.3
Button name:  O3
Value:  17.1
append
Button name:  NO2
Value:  66.7
Button name:  SO2
Value:  0
Button name:  CO
Value:  57.3


City:  Melbourne
Level:  Good
Button name:  PM2.5
Value:  1.9
Button name:  PM10
Value:  19.9
Button name:  O3
Value:  12.3
append
Button name:  NO2
Value:  3.6
Button name:  SO2
Value:  4.1
Button name:  CO
Value:  0


Finish


In [15]:
data = {'Level': status_text, 'PM2.5': PM25, 'PM10': PM10, 'O3': O3, 'NO2': NO2, 'SO2':SO2, 'CO': CO, 'City': city_names}
df = pd.DataFrame(data)


**Xem thử dataframe**

In [17]:
df

,Level,PM2.5,PM10,O3,NO2,SO2,CO,City
0,Hazardous,337.4,483.6,31.8,67.4,22.2,2880.9,Delhi
1,Hazardous,286.5,2.2,NULL,NULL,NULL,NULL,Bishkek
2,Hazardous,227.4,NULL,NULL,NULL,NULL,NULL,Lahore
3,Very Unhealthy,148.2,NULL,NULL,NULL,NULL,NULL,Dhaka
4,Very Unhealthy,118.1,201.7,56.9,172,13.8,1487.5,Guangzhou
...,...,...,...,...,...,...,...,...
90,Good,10.3,12.7,34.8,15.5,1.8,NULL,Skopje
91,Good,6.2,NULL,2.4,2.6,0.7,17.1,Portland
92,Good,3.2,4.2,30,24.7,1,251.2,Vancouver BC
93,Good,1.3,NULL,17.1,66.7,0,57.3,Salt Lake City


In [18]:
df.to_csv("data_8.csv")